In [1]:
import pandas as pd

from ddeserts.annotate import add_dvap_columns
from ddeserts.annotate import add_other_race_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.load import load_cvap_data

In [2]:
df = load_cvap_data('State')

In [3]:
# add extra columns
add_other_race_columns(df)
add_dvap_columns(df)
df = with_columns_sorted(df)
df

AttributeError: 'str' object has no attribute 'apply'

In [ ]:
df.to_csv('output/states.csv')